In [ ]:
# Import des modules
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import statsmodels.api as sm

In [ ]:
# Récuperer les données
df = pd.read_csv('./Data/Donnees_Quotidiennes.csv', index_col= 0,parse_dates=['Date'],sep=';');

In [ ]:
# Définir le période de traitement, ici quotidien
periods = dict([('W',52),('D',365),('S',4),('M',12)])
PeriodName = 'D'
PeriodValue = periods[PeriodName]

In [ ]:
# suppression des champs inutiles.
df  = df[['Code INSEE région','Région','Date','Consommation (MW)', 'Température (°C)']]
# Supprimer les lignes avec des valeurs absents
df.dropna(axis = 0, subset=["Consommation (MW)"], inplace=True)

In [ ]:
# Adapter la base de données à notre frequence d'utilisation souhaitée pour les predictions
df = pd.DataFrame(df.set_index('Date').groupby(['Code INSEE région','Région'])['Consommation (MW)','Température (°C)'].resample(PeriodName).mean())

In [ ]:
# Prévisualisation
df = df.reset_index()
df.head()

In [ ]:
#Application d'un filtre sur la région pour ne travailler pour le moment que sur la région Ile de france afin de simplifier les tests.
df_idf = df[df['Code INSEE région']== 11]

In [ ]:
# suppression des devenus inutiles
df_idf.drop(['Code INSEE région', 'Région'],inplace=True,axis=1)

In [ ]:
df_idf['test'] = pd.to_datetime(df_idf['Date'], format="%Y%m%d", unit='D')

In [ ]:
# paramétrage de la date en tant qu'index.
df_idf.set_index('Date',inplace=True)

### Méthode Box-Jenkins

Il s'agit de notre méthode choisi pour la construction de nos modèles des séries chronologiques afin de réaliser notre projet de façon rapide, efficace et rigoureuse.
3 étapes :
+ Identification:
    - Explorer les données pour en trouver une forme appropriée à la modélisation ARIMA / ARIMAX / SARIMA / SARIMAX
        - Identifier si la série est stationnaire 
        - Trouver quelle transformation et / ou différenciation rendra la serié stationnaire - I
        - Identifier les ordres p, q, P et Q les plus prometteurs
        - Déterminer si la série chronologique es saisonnière, si c'est le cas, trouver sa période saisonnière
        - Outils : traçage des séries cronologiques, Test Dicky-Fuller, pour la tranformation / differenciation (df.diff(), np.log(), np.sqrt()), l'ACF et le PACF


+ Estimation :
    - Utiliser les méthodes numériques pour estimer les coefficients AR / MA 
    - Ajustement du modèle (model.fit())
    - Comparer plusieurs modeles en utilisant l'AIC et le BIC


+ Diagnostique du modèle :
    - Evaluer la qualité du modèle le mieux adapté :
        - Réaliser de tests statistiques pour s'assurer que les résidus se comportent bien (les résidus ne sont pas correlées, et ils suivent une distribution normale) 


En utilisant les informations recueilles à partir de tests statistiques et et celles pendant l'étape de diagnostic, nous devons prendre une décision :
- Le modèle est-il assez bon ou devons-nous revenir en arrièe et le retravailler ?
- Lors de l'étape du diagnostique, si les résidus ne sont pas comme ils devraient être, nous reviendrons et repenserons nos choix dans les étapes précédentes. Et si les résidus sont corrects, nous pouvons aller de l'avant et faire des prédictions


## Identification

In [ ]:
fig, ax = plt.subplots()
df_idf['Consommation (MW)'].plot(ax=ax)
plt.show()

Nous constatons que notre série a des modèles qui se répètent à intervalles réguliers, synonime d'une série saisonnière.

Afin de continuer, nous devons séparer notre série en un jeu d'entrainement et de test.
Ici, nous utilisons les valeurs passées pour faire des prédictions futures, et nous devrons donc diviser les données dans le temps

In [ ]:
#Séparation en un jeu d'entrainement et de test
train=df_idf[df_idf.index < '2020-11-21']
test=df_idf[df_idf.index > '2020-11-20']
train.columns

In [ ]:
# Séparation des données meteo et conso
train_conso = train['Consommation (MW)']
train_meteo = train['Température (°C)']
test_conso = test['Consommation (MW)']
test_meteo = test['Température (°C)']

Pour modéliser notre série chonologique, elle doit être stationnaire, c'est-à-dire que la distribution des données ne change pas avec le temps.

Pour cela, nous devons vérifier que :
+ la série n'a aucune tendance
+ la variance est constante (la distance moyenne des points de données de la ligne 0 ne change pas)
+ L'autocorrélation est constante

In [ ]:
#function qui permet de tester si les données sont stationnaires.

def get_stationarity(timeseries):
    
    # Statistiques mobiles
    rolling_mean = timeseries.rolling(window=PeriodValue).mean()
    rolling_std = timeseries.rolling(window=PeriodValue).std()
    
    # tracé statistiques mobiles
    original = plt.plot(timeseries, color='blue', label='Origine')
    mean = plt.plot(rolling_mean, color='red', label='Moyenne Mobile')
    std = plt.plot(rolling_std, color='black', label='Ecart-type Mobile')
    plt.legend(loc='best')
    plt.title('Moyenne et écart-type Mobiles')
    plt.show(block=False)
    
    # Test Dickey–Fuller :
    result = adfuller(timeseries, regression='ct')
    print('Statistiques ADF : {}'.format(result[0]))
    print('p-value : {}'.format(result[1]))
    print('Valeurs Critiques :')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

In [ ]:
# Tester la série temporelle
get_stationarity(train_conso)

La moyenne mobile a une legere tendance à la baisse, il existe alors une tendance negative.

Le Test Dickey–Fuller nous donne un p-value inférieur à 0.05, on rejet H0 et on retient alors l'hypothese de stationarité. 
Neamoins, dû à la présence de tendance, nous devons alors proceder à la differenciation de la série

In [ ]:
# Effectuer la differenciation de la série, puis supprimer les Nan
train_stationary = train_conso.diff().dropna()

# Tester la stationnarité de notre nouvelle série
get_stationarity(train_stationary)

* H0:la série a une racine unitaire, ce qui indiquerait qu'elle est non stationnaire.
* H1: H0 est rejetée et la série n'a pas de structure temporellement dépendante : elle est stationnaire.

Nous utiliserons la p-valeur de ce test pour déterminer l'hypothèse à retenir au seuil de 5%.
* p valeu > 0.05 : H0 n'est pas rejetée. La série n'est pas stationnaire
* p valeu ≤ 0.05 : H0 est rejetée et la série est stationnaire.

Avec une valeur critique de 5% et un p-value de 5.08e-10, (p valeu ≤ 0.05) nous rejettons notre Hyphothese nulle, donc notre série est stationnaire.
Nous constantons aussi que la moyenne Mobile n'a plus de tendance

Maintenant, nous utiliserons la méthode seasonal_decompose afin d'analyser la composante saisonnière, confirmer l'absence de tendance et le résiduel.

Alors, pour décomposer les données, nous devons savoir à quelle fréquence les cycles se répètent

In [ ]:
# Trouver les périods / fréquence
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

fig, ax = plt.subplots(figsize= (17,5))
plot_acf(train_conso, lags = 730, ax=ax, zero=False)
plt.show()

Nous pouvons clairement voir qu'il y a une période saisonnière de 365 étapes

Maintenant, nous devons trouver l'ordre de différenciation.
Pour rendre une série chronologique stationnaire, nous pouvons avoir besoin d'appliquer une différenciation saisonnière (au lieu de soustraire le plus récent valeur de la série chronologique, nous soustrayons la valeur de série chronologique d'un cycle antérieur)

In [ ]:
# le parametre Period est à modifier selon la frequence determinée par l'ACF
# [('W',52),('D',365),('S',4),('M',12)]
# On applique la méthode seasonal_decompose à notre série d'entraînement
res = sm.tsa.seasonal_decompose(train_conso,period=365)
fig = res.plot()
fig.set_figheight(5)
fig.set_figwidth(10)
plt.show()


Cette décomposition nous permet d'identifier une tendance descroissante linéairement, ainsi qu'une saisonnalité de période 12 (annuelle)

In [ ]:
# On applique la méthode seasonal_decompose à notre série d'entraînement différenciée
res = sm.tsa.seasonal_decompose(train_stationary,period=365)
fig = res.plot()
fig.set_figheight(5)
fig.set_figwidth(10)
plt.show()

On observe une tendance plutot constant, mais des résidus avec trop de variations en fonction du temps

Nous constatons un cycle saisonnier, alors nous prendrons la différence saisonnière.

à retenir : Jamais faire plus de deux ordres de différenciation au total

Une fois que nous avons trouvé les deux ordres de différenciation, et fait les séries chronologiques stationnaires, nous devons trouver les autres paramètres

In [ ]:
# Différence 
# diff(1), différenciation "d"
# diff(365), longueur du cycle saisonnier, différenciation saisonniière "D"
train_stationary_s = train_conso.diff().diff(365).dropna()

### Identifier les ordres p,q non saisonnières et les ordres P,Q saisonnières les plus prometteurs

In [ ]:
# Tracer la fonction d'autocorrélation simple et la fonction d'autocorrélation partielle 
# de la série différenciée
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plt.figure(figsize= (17,5))
plt.subplot(121)
plot_acf(train_stationary, lags = 50, zero = False, ax=plt.gca())
plt.subplot(122)
plot_pacf(train_stationary, lags = 50, zero = False, ax=plt.gca())
plt.show()

La zone bleutée représente la zone de non significativité des autocorrélogrammes. 
Les graphiques de l'ACF et de la PACF indiquent un "p" = (0) et un "q" = (0)  

Pour rappel, on se base ici sur le fait que dans un modèle  AR(p) l'autocorrélogramme simple s'annule au rang  p+1 et pour un modèle  MA(q) l'autocorrélogramme partiel s'annule au rang  q+1 .

In [ ]:
# Tracer la fonction d'autocorrélation simple et la fonction d'autocorrélation partielle 
# de la série différenciée s

# Liste de lags, corresponde à la frequence avec laquelle un evenetment se repete, chaque 365 etapes
lags = [1,365,730]

plt.figure(figsize= (17,5))
plt.subplot(121)
plot_acf(train_stationary_s, lags = lags, zero = False, ax=plt.gca())
plt.subplot(122)
plot_pacf(train_stationary_s, lags = lags, zero = False, ax=plt.gca())
plt.show()


La zone bleutée représente la zone de non significativité des autocorrélogrammes. 
Les graphiques de l'ACF et de la PACF indiquent un "P" = (1) et un "Q" aussi = (0)  


*On va donc fitter un modèle SARIMA(0,1,0)(1,1,0,7) + la variable externe : Temperature*


## Estimation

In [ ]:
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
sns.set(font='IPAGothic')
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
sarimax_model = SARIMAX(train_conso, order=(0,1,0), seasonal_order=(1,1,0,7), exog=train_meteo,
                enforce_stationarity=False, enforce_invertibility=False, freq='D', trend='n')
res = sarimax_model.fit()
res.summary()

Nous allons utiliser l'outil d'evaluation "Critére d'Information Akaike - AIC", afin d'évaluer la bonne adéquation du modèle pour les predictions et comparer aussi plusieurs modèles.

Rappel :
* Un modèle qui fait des meilleurs prévisions reçoit un score AIC inférieur
* AIC privilege les modèles simples, donc moins de parametres

L'outil "BIC - Critère d'Information Bayésien" est similaire à l'AIC, mais il est utilisé pour trouver le meilleur modèle explicative

Afin de vérifier que les paramtres choisis rendrent le meilleur modèle, nous allons tester d'autres parametres afin de trouver le meilleur score AIC

In [ ]:
import pmdarima as pm

results = pm.auto_arima(train_conso,
                        d= 1,                        # Ordre de différenciation non saisonnière
                        max_p = 3,
                        max_q = 3,
                        seasonal = True,             # Car notre serie est bien saisonnière
                        stationary=False,            # Notre sèrie n'est pas stationnaire
                        m= 7,                        # périod saisonnière
                        D = 1,                       # Ordre de différenciation saisonnière
                        max_P = 3,
                        max_Q = 3,
                        information_criterion='aic', # Afin de choisir le meilleur modèle basé sur l'AIC
                        trace = True,                # afficher l'AIC du modèle
                        error_action = 'ignore',     # ignorer les ordres qui ne marchent pas
                        stepwise = True,             # méthode de recherche intelligente
                        trend=None,                  # Tendance
                        scoring='mse')

La méthode auto_arima nous indique que le meilleur modèle serait un SARIMAX (3,1,0)(3,1,0)[7]

In [ ]:
# Afficher les résultats du modèle SARIMAX
print(results.summary())

## Diagnostique du modèle

- Pour diagnostiquer notre modèle, nous allons analyser les résidus des données d'entraînement.
- Les résidus sont la différence entre nos modèles de prediction et les valeurs réelles de la série
- Pour un modèle idéal, les résidus doivent être du bruit blanc (chaque valeur n'est pas corrélée avec les valeurs précédentes), non corrélé centré sur 0

In [ ]:
# Analyser les résidus du model avec plot_diagnostics
sns.set_style("white")
results.plot_diagnostics()
plt.show()

L'histogramme nous montre la distribution mesurée, ligne verte montre une distribution normale et la ligne orange montre une version lisée de cet histogramme. 
Si notre modèle est bon, ces deux lignes devraient être presque identiques. Ce n'est pas exactement le cas pour notre modèle.

La graphique Q-Q Normal, est une autre façon de montrer la distribution des résidus du modèle et vérifier s'ils suivent une distribution normale. Pour cela, les points blues doivent se trouver au tour de la ligne rouge. Ce n'est pas exactement le cas pour notre modèle.

Le correlogramme est une trace  ACF des résidus, 95% des corrélations pour un décalage supérieur à zéro de devraient pas être significatives. S'il y a une corrélation significative dans les résidus, cela signifie qu'il y a des informations dans les données que notre modèle n'a pas saisies.

- Prob(Q) est la valeur de p associée à l'hypothèse nulle selon laquelle les résidus n'ont pas de structure de corrélation
- Prob(JB) est la valeur de p associée à l'hypothèse nulle selon laquelle les résidus suivent une distribution normale

Si une des ces valeurs est inférieur à 0.05 nous rejetons l'hypothèse
* p valeu > 0.05 : H0 n'est pas rejetée. 
* p valeu ≤ 0.05 : H0 est rejetée

Ici, avec Prob(Q) = 0.47 nous ne rejetons pas H0, cela signifie que nos résidus n'ont pas de structure de corrélation 
Par contre, nous rejetons H0 pour Prob(JB), donc les résidus ne suivent pas une distribution normale

Il semble que nous devons faire quelques ajustements sur les parametres de différenciation afin de trouver un bon modèle ou trouver un autre moyen pour transformer la série

In [ ]:
#### Avec un modèle SARIMAX (3,1,0)(3,1,0)[7]
model = SARIMAX(train_conso, order=(3,1,0), seasonal_order=(3,1,0,7), exog=train_meteo, 
                enforce_stationarity=False, enforce_invertibility=False, freq='D', trend='n').fit()

# "Steps" concerne le nombre de predictions à efectuer
forecast = model.get_forecast(steps=10, exog = test_meteo)

# Determiner nos intervalles de confiance
mean_forecast = forecast.predicted_mean
confidence_intervals = forecast.conf_int()
lower_limits = confidence_intervals['lower Consommation (MW)']
upper_limits = confidence_intervals['upper Consommation (MW)']


In [ ]:
matrice = pd.DataFrame({'Reel': test['Consommation (MW)'], 
            'predits' : mean_forecast.values,
            'lower_limits' : lower_limits,
            'upper_limits' : upper_limits,
            'T' : test['Température (°C)'] }, index = test.index)
matrice.head()

In [ ]:
#Graphique
font1 = {'family':'Times','color':'darkblue','size':18}
font2 = {'family':'Times','color':'black','size':14}

sns.set_style("white")

plt.figure(figsize=(12, 8))
plt.plot(test_conso.index, test_conso, label='Valeurs réellees', color='blue', marker='o',linestyle='-.')
plt.plot(mean_forecast.index, mean_forecast.values, label="Valeurs predites", color='red', marker='o',linestyle='-.')
plt.fill_between(mean_forecast.index, lower_limits, upper_limits, color='pink')
plt.legend(labelcolor='markerfacecolor')
plt.title("Prédiction avec le modèle SARIMAX (3,1,0)(3,1,0)[7] Quotidien", fontdict = font1)
plt.xlabel('Date', fontdict = font2)
plt.ylabel('Moyenne de Consommation (MW)', fontdict = font2)
plt.xticks(rotation=45)
#plt.grid(False)
plt.show()

In [ ]:
from math import pi
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import Legend, DatetimeTickFormatter, formatters, HoverTool, LinearAxis, Range1d

output_notebook()

In [ ]:
#Source
source = ColumnDataSource(matrice)

# List de tools
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset"

p = figure(plot_width = 600, plot_height = 400,     
           title = "Prédiction avec le modèle SARIMAX (3,1,0)(3,1,0)[7] Quotidien",                    
           x_axis_label = 'Date', x_axis_type="datetime", y_axis_label ='Moyenne de Consommation (MW)',
           tools=TOOLS)


p.title.text_color = "darkblue"
p.title.text_font = "times"
p.title.text_font_size = "20px"
p.title.align = 'center'


p.varea(x = 'Date', y1 = 'lower_limits', y2='upper_limits', color='pink', alpha=0.5, source = source)

p.line(x = 'Date', y='Reel', color = "navy", legend_label = "Valeurs réellees", source = source)   
p.circle(x = 'Date', y='Reel', color = "navy",fill_color='white', size=8, source = source)

p.line(x = 'Date', y='predits', color = "red", legend_label = "Valeurs predites", source = source) 
p.circle(x = 'Date', y='predits', color = "red", fill_color='white',size=8, source = source)

p.xaxis.major_label_orientation = pi/4
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.ticker.desired_num_ticks = 10


# Activation de l'interaction avec la légende
p.legend.location = "top_left"
p.legend.click_policy = 'hide'

# Style hover
p.add_tools(HoverTool(
    tooltips=[('Date', '@Date{%Y-%m-%d}'),
        ('Prédiction', '@predits{0.00}'),
        ('Valeur réelle', '@Reel{0.00}')],formatters={'@Date': 'datetime'}))

show(p);

Le nivel de prediction n'est pas loin des valeurs réelles, neamoins nous avons constaté que les résidus n'avaient pas une distribution normale, c'est à dire que les résidus n'arrivaient pas à expliquer les variations du modèle, ce qui nous pouvons constater sur le graphique. 



In [ ]:
#Source
source = ColumnDataSource(matrice)


# List de tools
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset"


y_overlimit = 0.05 
p = figure(plot_width = 600, plot_height = 400,     
           title = "Prédiction avec le modèle Elastic Net Quotidien",                    
           x_axis_label = 'Date', x_axis_type="datetime",
           y_axis_label = 'Consommation Moyenne',
           toolbar_location="below",
           tools=TOOLS)  


p.title.text_color = "darkblue"
p.title.text_font = "times"
p.title.text_font_size = "20px"
p.title.align = 'center'

p.varea(x = 'Date', y1 = 'lower_limits', y2='upper_limits', color='pink', alpha=0.5, source = source)

p.line(x='Date', y = 'Reel', color = "navy", legend_label = "Valeurs réellees", source = source)   
p.circle(x='Date', y ='Reel', color = "navy",fill_color='white', size=8, source = source)

p.line(x='Date', y ='predits', color = "red", legend_label = "Valeurs predites", source = source) 
p.circle(x='Date', y='predits', color = "red", fill_color='white',size=8, source = source)

# axis y, gauche
p.y_range = Range1d(matrice['Reel'].min() * (1 - y_overlimit), matrice['Reel'].max() * (1 + y_overlimit))

p.xaxis.major_label_orientation = pi/4
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.ticker.desired_num_ticks = 10

# Axis y, droite
y_column2_range = "T" + "_range"
p.extra_y_ranges = {
    y_column2_range: Range1d(
        start=matrice['T'].min() * (1 - y_overlimit),
        end=matrice['T'].max() * (1 + y_overlimit),
    )
}
p.add_layout(LinearAxis(y_range_name=y_column2_range), "right")

p.line( x='Date', y = 'T', color="grey", legend_label="T (C°)", y_range_name=y_column2_range, source = source)
p.circle(x='Date', y = 'T', color = "grey",fill_color='white', size=8, y_range_name=y_column2_range, source = source)


# Activation de l'interaction avec la légende
p.legend.location = "top_center"
p.legend.click_policy = 'hide'

# Style hover
p.add_tools(HoverTool(
    tooltips=[('Date', '@Date{%Y-%m-%d}'),
        ('Prédiction', '@predits{0.00}'),
        ('Valeur réelle', '@Reel{0.00}'),
        ('C°', "@T{0.00}")],
    formatters={'@Date': 'datetime'}
))

show(p);


In [ ]:
# Calculer et afficher le score MAPE
y_true, y_pred = np.array(test['Consommation (MW)']), np.array(mean_forecast)
MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("Mean Absolute Prediction Error : %0.2f%%"% MAPE)

Avec notre Modèle ARIMA(3,1,0)(3,1,0)[7] nous avons une probabilité moyenne d'erreur de **4.41%**

In [ ]:
# Racine carrée de la Moyenne des résidus au carré.
# RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse(mean_forecast, test['Consommation (MW)'])

plus l'erreur quadratique moyenne est proche de 0, plus précises sont les prédictions.

Nous pourrons donc comparer ce résultat avec le RMSE des autres modèles

In [ ]:
#MSE
from sklearn.metrics import mean_squared_error

mean_squared_error(test,y_pred)